In [ ]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
model = pybamm.BaseModel("height model")

# define parameters
w = pybamm.Parameter("Coating width [m]")

# define variable
h = pybamm.Variable("Coating thickness [m]")
u_w = pybamm.Variable("Web speed [ms-1]")
v = pybamm.Variable("Volume flow [m3s-1]")

# governing equations
h = v / w / u_w 
